In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
# import os
# module_a_dir = os.path.dirname(os.path.abspath(__file__))
# utils_path = os.path.join(module_a_dir, '..', 'Utils')
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import validation_data as Val
import var_A_x_B as vAB
import utils as uti

sys.path.append(workdir_ + 'PyRegridding/Utils/')
import MakePressures as MkP
import GridUtils as GrU


sys.path.append(workdir_ + 'PyRegridding/Drivers/')
import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


In [ ]:
### Make a list of files

In [ ]:
%%time

Dst='fv1x1'

"""
#exp='c64_gwR2_ne30pg3_FMTHIST_L120_topfix_oldCTL'
exp='c64_gwR2_ne30pg3_FMTHIST_topfix_oldCTL_qbotune_x03'
subd, Src, Hkey='hist','ne30pg3','c'
useri='juliob'
Dst='fv1x1'
"""

"""
exp='f.e30_cam6_4_036.FMTHIST.ne120_L93.001'
subd, Src, Hkey='hist','fvQxQ','yx'
useri='hannay'
Dst='fvQxQ'
"""

"""
exp='c153_topfix_ne120pg3_FMTHIST_xic_x02'
subd, Src, Hkey='hist','ne120pg3','c'
useri='juliob'
Dst='fvQxQ'
"""
"""
exp='c153_topfix_ne240pg3_FMTHIST_xic_x01'
subd, Src, Hkey='hist','ne240pg3','c'
useri='juliob'
Dst='latlonOxO'
"""
#"""
#exp='c153_topfix_ne240pg3_FMTHIST_L56_86km_xic_x02'
exp='c153_topfix_ne240pg3_FMTHIST_xic_x02'
subd, Src, Hkey='hist','ne240pg3','c'
useri='juliob'
Dst='latlonOxO'
#"""

ymdPat='*'

#ymdPat=['2002-*'] #'2003-*' ]
"""
ymdPat=['2000-*']
ymdPat=['2001-*' ]
ymdPat=['2002-*' ]
"""


A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat='cam.h0a' , ymdPat=ymdPat,verbose=True, open_dataset=True )

exp_A=A.exp
exp_Val='ERA5'


In [ ]:
########################################
# Make seasonal means
########################################

print( f'{A.exp} Years {len(A.X.time) /12. }')

season='jja'
UUa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='U',return_time=True)
TTa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='T',return_time=True)


In [ ]:
########################################
# Make zonal means
########################################
method='C2'
wgtFile=f"/glade/work/juliob/GridFiles/Weights/{Src}_x_{Dst}_{method}_wgts.nc"
print(wgtFile)

if (Hkey == 'yx' ):
    ########################
    # Option 1 from lat-lon regirrdded data
    ########################
    
    
    
    #ps_x =Dx.PS.values
    lon_a=A.X.lon.values
    lat_a=A.X.lat.values
    lev_a=A.X.lev.values
    zlev_a=-7.0*np.log( lev_a/1_000. )
    # Zonal Av
    UUa_sz=np.average( UUa_s , axis=2 )
    TTa_sz=np.average( TTa_s , axis=2 )

elif (Hkey=='c'):
    ########################
    # Option 2 from unstructured SE data
    ########################
    
    
    DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
    lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
    lev_a=A.X.lev.values
    zlev_a=-7.0*np.log( lev_a/1_000. )
    
    #RegridObj = RgF.Horz(Src=Src, Dst=Dst )
    UUa_s_xy=RgF.Horz(xfld_Src=UUa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
    TTa_s_xy=RgF.Horz(xfld_Src=TTa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
    
    # Zonal Av
    UUa_sz=np.average( UUa_s_xy , axis=2 )
    TTa_sz=np.average( TTa_s_xy , axis=2 )


In [ ]:
####
# Validation
####
Validation_on_control_mgrid = False

if (Validation_on_control_mgrid == True):
    PSc_s = Av.Seasonal( ds=Dc, season=season , fld='PS')
    mgrid={'ps':PSc_s, 'hyam':hyam_c, 'hybm':hybm_c, 'hyai':hyai_c, 'hybi':hybi_c, 'hgrid':'fv0.9x1.25' }
else:
    mgrid=False

ERAyears = '2004' #'*' #'2010' #'*' #'2000'

Uval = Val.data(fld='U',season=season,mgrid=mgrid,zlev=True , Years=ERAyears ) #, Years='1996' ) #, Years='2000' )
UUv_s , zlev_v,lat_v,lon_v, years_Uv = Uval['aa'], Uval['lev'], Uval['lat'], Uval['lon'], Uval['years']
UUv_sz=np.average( UUv_s , axis=2 )

Tval = Val.data(fld='T',season=season,mgrid=mgrid,zlev=True , Years=ERAyears ) #, Years='1996' ) #, Years='2000' )
TTv_s , zlev_v,lat_v,lon_v, years_Tv = Tval['aa'], Tval['lev'], Tval['lat'], Tval['lon'], Tval['years']
TTv_sz=np.average( TTv_s , axis=2 )



In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))


# Create 2D arrays
Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
nxplo,nyplo=2,2
# Adding the hatching for the sponge layer
sponge_layer = np.zeros_like(UUa_sz)
sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1


fig = plt.figure(figsize=(15, 12))


ulev=np.linspace( -60,140,num=21) 
tlev=np.linspace( 180,300,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

titlesize=18

"""
titles = [ f"Control: $\Delta x$=100km (ne30) {season.upper()}" ,
          f"$\Delta x$=28km (ne120) {season.upper()} " ,
          f"$\Delta x$=14km (ne240) {season.upper()} " ,
          f"Validation <{exp_Val}> {season.upper()}  " ]
"""
titles = [ f"<{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}" ,
          f"<{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}" ,
          f"<{exp_Val}> {season.upper()} {years_Uv} " ,
          f"<{exp_Val}> {season.upper()} {years_Tv} " ]
plotTop=82

n=1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,UUa_sz ,levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,UUa_sz ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)


n=3
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=ulev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=ulev, colors='black')
#ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_title( titles[n-1] ,fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

n=2
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,TTa_sz ,levels=tlev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,TTa_sz ,levels=tlev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)


n=4
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, TTv_sz ,levels=tlev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, TTv_sz ,levels=tlev, colors='black')
#ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_title( titles[n-1] ,fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')


FigName = f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
print( FigName) 
Pu.save_link(png_filename = FigName )

